## Falcons code by Yusuke modified to calculate non linear cross link terms

In [1]:
#using PyPlot
#include("/gpfs/home/cmb/yusuket/program/scan_strategy/optimisation2/functions/spin_characterisation_w_HWP.jl")
#data = npzread("/home/cmb/yusuket/program/scan_strategy/_optimisation/spin_HWP/nside512_1yr_19Hz_46rpm_0.1arcmin.npz")
#lbt  = pyimport("lbsim_tools")
#cmap = lbt.get_planck_cmap()
using Healpix
using Falcons
using PyCall
using PyPlot
using ProgressMeter
using DataFrames
hp = pyimport("healpy");

In [2]:
#questo calcola matr cross link normale 
function get_scanfield_2(ss::ScanningStrategy,; division, spin_n, spin_m)
    #println("RUN `get_scanfield`")
    orientation_func_hwp(n, m, ψⱼ, ϕⱼ) = ℯ^(-im*(n*ψⱼ + m*ϕⱼ))
    h      = orientation_func_hwp
    resol  = Resolution(ss.nside)
    npix   = nside2npix(ss.nside)
    chunk  = Int(ss.duration / division)
    ω_hwp  = rpm2angfreq(ss.hwp_rpm)
    hitmap = zeros(Int64, npix)
    #hₙₘ     = zeros(Complex{Float32}, (length(spin_n), length(spin_m), npix))
    hₙₘ    = zeros(Complex{Float32}, (18, npix))
    BEGIN  = 0
    progress = Progress(division)
    @views @inbounds for i = 1:division
        END = i * chunk
        theta, phi, psi, time = get_pointings(ss, BEGIN, END)
        @views @inbounds for j = eachindex(ss.quat)
            theta_j = theta[:,j]
            phi_j   = phi[:,j]
            psi_j   = psi[:,j]
            polang  = get_pol_angle(ss, j)
            @views @inbounds for k = eachindex(time)
                t = time[k]
                p = pointings(resol, theta_j[k], phi_j[k], psi_j[k], mod2pi(ω_hwp*t)+polang)
                χ = mod2pi(2ω_hwp*t - p.ψ)
                newangle = mod2pi(2*ω_hwp*t) # match this angle with TOD angles in lbsim --> checked
                
                hitmap[p.Ω] += 1
                               
                 
                hₙₘ[1, p.Ω] += cos(2χ)
                hₙₘ[2, p.Ω] += sin(2χ)
                hₙₘ[3, p.Ω] += cos(2χ)*cos(2χ)
                hₙₘ[4, p.Ω] += sin(2χ)*cos(2χ)
                hₙₘ[5, p.Ω] += sin(2χ)*sin(2χ)
                
                hₙₘ[6, p.Ω] += cos(2χ)*cos(2χ)*cos(2χ)
                hₙₘ[7, p.Ω] += sin(2χ)*sin(2χ)*cos(2χ)
                hₙₘ[8, p.Ω] += sin(2χ)*cos(2χ)*cos(2χ)
                hₙₘ[9, p.Ω] += sin(2χ)*sin(2χ)*sin(2χ)
                
                hₙₘ[10, p.Ω] += cos(newangle)
                hₙₘ[11, p.Ω] += cos(newangle)*cos(2χ)
                hₙₘ[12, p.Ω] += cos(newangle)*cos(2χ)*cos(2χ)
                hₙₘ[13, p.Ω] += cos(newangle)*sin(2χ)
                hₙₘ[14, p.Ω] += cos(newangle)*sin(2χ)*sin(2χ)
                hₙₘ[15, p.Ω] += cos(newangle)*cos(newangle)
                hₙₘ[16, p.Ω] += cos(newangle)*cos(newangle)*cos(2χ)
                hₙₘ[17, p.Ω] += cos(newangle)*sin(2χ)*cos(2χ)
                hₙₘ[18, p.Ω] += cos(newangle)*cos(newangle)*sin(2χ)
                
                #=
                @views @inbounds for _n in eachindex(spin_n)
                    @views @inbounds for _m in eachindex(spin_m)
                        hₙₘ[_n, _m, p.Ω] += h(spin_n[_n], spin_m[_m], p.ψ, p.ϕ)
                    end
                end
                =#
            end
        end
        BEGIN = END
        next!(progress)
    end
    #=
    @views for _n in eachindex(spin_n)
        @views for _m in eachindex(spin_m)
            hₙₘ[_n,_m,:] ./= hitmap
        end
    end
    df = get_hnm_quantify(hₙₘ, spin_n, spin_m)
    =#
    for _n in 1:18
        hₙₘ[_n,:] ./= hitmap
    end
    df = DataFrame()
    return scanfield(hitmap, hₙₘ, df, spin_n, spin_m, ss)
end

get_scanfield_2 (generic function with 1 method)

In [3]:
ss = gen_ScanningStrategy()
ss.nside         = 64
ss.hwp_rpm       = 39.
ss.sampling_rate = 19.
ss.duration      = 1*365*60*60*24
ss.coord         = "G"
#ss.start_angle   = deg2rad(0)


division         = 365*12*12*24
spin_n           = -6:6   |> collect #np.arange(-6,6)
spin_m           = -8:4:8 |> collect; #[-8,-4,0,4,8]

In [4]:
ss.quat

1-element Vector{Vector{Float64}}:
 [0.0, 0.0, 1.0, 0.0]

In [5]:
## block to chose which dets

imo_path         = "/Users/silviamicheli/Desktop/lb_sim/litebird_sim/litebird_imo/IMO/schema.json"
#base_path        = "/group/cmb/litebird/usr/ytakase/pointing_error/pointing_sys_falcons"
#target_dir       = "/gpfs/home/cmb/yusuket/litebird/pointing_sys_casex"
#imo_path         = "/Users/silviamicheli/Downloads/schema_IMo_v2.1_2023_11_30.json"
filename         = "./detectors_MFT_M1-140.txt" 
imo              = gen_imo(imo_path);

telescope        = split.(filename, "_")[2] |> String
channel_name     = split.(filename, "_")[3] |> String

inst_info        = get_instrument_info(imo, telescope)
channel_list     = get_channel_list(imo)
channel_info     = get_channel_info(imo, channel_name)
#file_path        = target_dir * filename
bolonames        = get_detectors(filename)
imo_name!(ss, imo, name=bolonames)
show_ss(ss)

nside                    : 64 
duration [sec]           : 31536000.0 
sampling rate [Hz]       : 19.0 
alpha [deg]              : 45.0 
beta [deg]               : 50.0 
gamma [deg]              : 0.0 
prec. period [min]       : 192.348
↳ prec. rate [rpm]       : 0.005199
spin period [min]        : 20.000
↳ spin rate [rpm]        : 0.050000
HWP rot. rate[rpm]       : 39.000000 
start angle              : 0.000000 
coordinate system        : G 
FPU                     
↳ Det. 1  name | 001_004_030_00A_140_T              : (x,y,z,w) = [0.075, 0.010, 0.382, 0.921] 
↳ Det. 2  name | 001_004_030_00A_140_B              : (x,y,z,w) = [0.060, -0.046, 0.921, 0.382] 
↳ Det. 3  name | 001_002_030_00A_140_T              : (x,y,z,w) = [-0.075, -0.010, 0.382, 0.921] 
↳ Det. 4  name | 001_002_030_00A_140_B              : (x,y,z,w) = [-0.060, 0.046, 0.921, 0.382] 


In [6]:
field2 = get_scanfield_2(ss, division=division, spin_n=spin_n, spin_m=spin_m);

Progress: 100%|█████████████████████████████████████████| Time: 5:33:22


In [7]:
hp.write_map("clinks_maps/4testfull_a.fits",real.(field2.h[1,:]))
hp.write_map("clinks_maps/4testfull_b.fits",real.(field2.h[2,:]))
hp.write_map("clinks_maps/4testfull_c.fits",real.(field2.h[3,:]))
hp.write_map("clinks_maps/4testfull_d.fits",real.(field2.h[4,:]))
hp.write_map("clinks_maps/4testfull_e.fits",real.(field2.h[5,:]))
                    
hp.write_map("clinks_maps/4testfull_f.fits",real.(field2.h[6,:]))
hp.write_map("clinks_maps/4testfull_g.fits",real.(field2.h[7,:]))
hp.write_map("clinks_maps/4testfull_h.fits",real.(field2.h[8,:]))
hp.write_map("clinks_maps/4testfull_j.fits",real.(field2.h[9,:]))
                                    
hp.write_map("clinks_maps/4testfull_mu.fits",real.(field2.h[10,:]))
hp.write_map("clinks_maps/4testfull_gamma.fits",real.(field2.h[11,:]))
hp.write_map("clinks_maps/4testfull_eps.fits",real.(field2.h[12,:]))
hp.write_map("clinks_maps/4testfull_eta.fits",real.(field2.h[13,:]))
hp.write_map("clinks_maps/4testfull_lambda.fits",real.(field2.h[14,:]))
hp.write_map("clinks_maps/4testfull_nu.fits",real.(field2.h[15,:]))
hp.write_map("clinks_maps/4testfull_delta.fits",real.(field2.h[16,:]))
hp.write_map("clinks_maps/4testfull_zeta.fits",real.(field2.h[17,:]))
hp.write_map("clinks_maps/4testfull_kappa.fits",real.(field2.h[18,:]))

setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
setting the output map dtype to [dtype('float32')]
